This notebook will pull from the FPL API the weekly results for each player and store it in a dictionary

Update weekly

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
top500 = pd.read_csv('top500.csv')
top500.head()

,id_,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,166,2,13,262,NaN,NaN,3,101668,1,-1,...,323.0,74,1236628,361992,540734,3203,1.1,8.0,Vardy,1
1,215,10,10,278,100.0,100.0,5,61366,1,-1,...,199.0,72,2759819,85655,1365443,11279,0.4,7.1,De Bruyne,0
2,192,3,8,185,100.0,100.0,3,110979,0,0,...,286.0,64,1738940,44324,710292,23502,0.5,5.4,Mané,0
3,214,1,7,223,NaN,NaN,4,103955,0,0,...,610.0,64,1472286,55954,1093035,15064,0.5,5.3,Sterling,3
4,210,3,6,258,NaN,NaN,2,37572,-1,1,...,392.0,62,1274982,12188,815242,164106,0.1,5.1,Agüero,0


In [3]:
histories = {}

ids = top500['id_']

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history   

In [20]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['gameweek'] = [i+1 for i in range((len(df.index)))]
    df['sum_points'] = [sum(df['total_points'].iloc[:i+1]) for i in range(len(df.index))]
    history_dfs[key] = df
    
    
    

In [21]:
type(history_dfs)

dict

In [22]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

(4880, 33)

In [23]:
top500histories.to_csv('top500histories.csv')